In [1]:
import pandas as pd
import re
import json


In [2]:
%matplotlib inline

In [3]:
df = pd.read_excel("./6417E_20161223150949.xls",header=3)
df.head()

,範本代碼,範本內容,更新日期,更新時間,人員代碼,人員代碼說明,異動人員,類別,類別說明,範本簡述
0,*a,Annn,20050624,144347,6417E,龔敏凱,6417E,D,醫師,NaN
1,*ap,Bil. apecies,20070407,121930,6417E,龔敏凱,6417E,D,醫師,NaN
2,*b,Bil.,20021001,170328,6417E,龔敏凱,6417E,D,醫師,NaN
3,*bap,Bil. apecies,20070616,173330,6417E,龔敏凱,6417E,D,醫師,NaN
4,*bl,both lung,20070313,204033,6417E,龔敏凱,6417E,D,醫師,NaN


In [4]:
df = df[[u'範本代碼', u'範本內容',u'更新日期']]

In [5]:
df.columns=['code','content','date']

In [6]:
df

,code,content,date
0,*a,Annn,20050624
1,*ap,Bil. apecies,20070407
2,*b,Bil.,20021001
3,*bap,Bil. apecies,20070616
4,*bl,both lung,20070313
5,*bll,Bil. lower lobes,20070721
6,*bul,BIL. upper lobes,20031015
7,*cp,central portion,20030121
8,*i,inner portion,20030121
9,*ip,inner portion,20030121


In [7]:
df = df.applymap(lambda x:str(x) )

In [8]:
df.date = pd.DatetimeIndex(df.date)

In [9]:
include = ['abdomen', 'sono', 'ultrasound']
exclude = ['breast', 'thyroid',]

def str_filter(w):
    return any(re.search(x,w,flags=re.IGNORECASE) != None for x in include) \
and not any(re.search(y,w,flags=re.IGNORECASE) != None for y in exclude)

def clean_text(t):
    t = re.sub(r"[Bb]il\.",r"bilateral",t,flags=re.MULTILINE )
    t = re.sub(r"[\n\r\s]+",r" ",t,flags=re.MULTILINE )
    t = re.sub(r":",r": \n",t,flags=re.MULTILINE )
    t = re.sub(r"(\D)\.","\g<1>.\n",t,flags=re.MULTILINE)
    t = re.sub(r"(\s+)(\d+)\.","\n\g<2>. ",t,flags=re.MULTILINE)
    t = re.sub(r"^\s+","",t ,flags=re.MULTILINE)
    return t

In [10]:
df.describe()

,code,content,date
count,13207,13207,13207
unique,13207,12364,2208
top,1u0011lnb,Sonography of the abdomen was performed and sh...,2013-01-18 00:00:00
freq,1,5,413
first,NaN,NaN,2002-07-08 00:00:00
last,NaN,NaN,2016-12-23 00:00:00


In [11]:
str_filter('SONOasda s abdomen ')

True

In [12]:
df[df.content.apply(lambda x: re.search(r'(abdomen)',x,flags=re.IGNORECASE) != None)]

,code,content,date
11,*la,lower abdomen,2007-03-13
13,*lla,left lower abdomen,2007-07-21
16,*lma,left middle abdomen,2007-07-22
30,*ra,right abdomen,2007-07-21
33,*rla,right lower abdomen,2007-04-08
38,*rua,right upper abdomen,2007-04-08
644,1flcds,Sonography of the abdomen was performed and sh...,2003-12-26
645,1flhkhar,Sonography of the abdomen was performed and sh...,2003-10-03
1266,alclhydr,Sonography of the abdomen was performed and sh...,2012-08-06
1267,alcrporcb,Sonography of the abdomen was performed and sh...,2015-06-10


In [13]:
result = df[df.content.apply(str_filter)]

In [14]:
result = result[result.code.apply(lambda x: x.startswith('s') )]

In [15]:
result.sort_values('date', ascending=False)

,code,content,date
8145,sgsrcsrca,Sonography of the abdomen was performed and sh...,2016-12-22
8139,sgsrcbrcar,Sonography of the abdomen was performed and sh...,2016-12-22
13116,sfllcrpo,Sonography of the abdomen was performed and sh...,2016-12-22
13117,sfllcrpopp,Sonography of the abdomen was performed and sh...,2016-12-21
11585,sfllcspoerrcs,Sonography of the abdomen was performed and sh...,2016-12-21
7077,sfllc2rahel,sfSonography evaluation of the abdomen was per...,2016-12-21
7274,sfllcbpoerrcb,Sonography of the abdomen was performed and sh...,2016-12-21
7516,sfllcpopp,Sonography of the abdomen was performed and sh...,2016-12-21
7308,sfllclhar,Sonography of the abdomen was performed and sh...,2016-12-20
6260,sflfslcrca,Sonography of the abdomen was performed and sh...,2016-12-20


In [16]:
cl = result.loc[result.code.apply(lambda x: len(x)).sort_values().index]
cl.content = cl.content.apply(clean_text)

In [17]:
#cl2 = cl[cl.code.apply(lambda x:len(x) < 6)]
cl2 = cl

In [18]:
text = ''
for i in cl2.itertuples(index=False):
    text += '[{}]:{}\n{}\n\n'.format(i.code, i.date,i.content)

In [19]:
with open('./new.txt','w') as f:
    f.write(text)

In [20]:
s = cl.code.apply(lambda x: x[1:])
l = s.tolist()

In [21]:
[i for i in l if len(i) in range(1,5)]

['s',
 'k',
 'g',
 'ca',
 'ck',
 'tk',
 'gs',
 'ap',
 'gb',
 'gw',
 'ci',
 'er',
 'k1',
 'kb',
 'rs',
 'ld',
 'lp',
 'tp',
 'lu',
 'li',
 'as',
 'pc',
 'an',
 'ng',
 'pd',
 'ph',
 'hp',
 'ha',
 'po',
 'ne',
 'nc',
 'pp',
 'ak',
 'rc',
 'he',
 'lm',
 'ch',
 'sk',
 'kr',
 'kp',
 'sb',
 'ab',
 'fl',
 'ah',
 'hs',
 'lc',
 'ke',
 'ad',
 'kl',
 'lmb',
 'lme',
 'har',
 'ldp',
 'has',
 'plc',
 'rca',
 'cie',
 'nfg',
 'pl2',
 'hal',
 'ner',
 'phe',
 'ffl',
 'rcl',
 'per',
 'rcr',
 'gss',
 'scr',
 'pel',
 'her',
 'ca2',
 'ltx',
 'cpo',
 'cbd',
 'nel',
 'ha2',
 'pne',
 'hyr',
 'akr',
 'po1',
 'po2',
 'fl1',
 'akl',
 'hel',
 'kel',
 'mcg',
 'mas',
 'pot',
 'keg',
 'luh',
 'hyl',
 'psp',
 'rc2',
 'plm',
 'hp2',
 'luc',
 'cas',
 'asp',
 'he2',
 'pos',
 'sgb',
 'flu',
 'pnc',
 'pcl',
 'jp3',
 'hpl',
 'rcs',
 'lcl',
 'hpc',
 'kbe',
 'kke',
 'kbf',
 'reb',
 'rpe',
 'lc2',
 'lcs',
 'rs2',
 'ark',
 'nvk',
 'lcr',
 'rsl',
 'app',
 'rsr',
 'kub',
 'rss',
 'kuf',
 'ld2',
 'kum',
 'sbi',
 'kbm',
 'tpo',
 'ld

In [22]:
K = cl[['code','content']]

In [23]:
K.code = K.code.apply(str.upper)

C:\Users\Falcon\Anaconda2\Lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [24]:
J = []
for i in K.itertuples():
   J.append([i.code, i.content])

In [279]:
json.dump(J,open('./../phrases.json','w'))

In [122]:
cl.content = cl.content.apply(clean_text)

In [153]:
breast = df[df.content.apply(lambda x :re.search('breast',x,flags=re.IGNORECASE) != None)]

In [155]:
breast.sort_values('date',ascending=False)

,code,content,date
1711,bmbcro,Bilateral screening mammogram: MLO and CC view...,2016-12-23
1717,bmbcsro,Bilateral screening mammogram: MLO and CC view...,2016-12-23
1704,bmbcbo,Bilateral screening mammogram: MLO and CC view...,2016-12-21
2060,bmo,Bilateral screening mammogram: MLO and CC view...,2016-12-21
13202,bm10,Bilateral mammogram: MLO and CC views.\r\n\r\n...,2016-12-16
13185,bm01o,Bilateral mammogram: MLO and CC views.\r\n\r\n...,2016-12-16
1692,bm10o,Bilateral mammogram: MLO and CC views.\r\n\r\n...,2016-12-16
2058,bmmwo,Bilateral screening mammogram: MLO and CC view...,2016-12-02
4096,mcmrn,Bilateral mammogram: MLO and CC views.\r\n\r\n...,2016-11-28
1475,bmmnobn,Bilateral mammogram: MLO and CC views.\r\n\r\n...,2016-10-20


In [25]:
J

[['SS', 'Suggest sonography.\n'],
 ['SK',
  'Sonography of the bilateral kidneys were performed and showed:\n1. The bilateral kidneys are normal configuration without hydronephrosis or stones.\n2. Right kidney about * mm in size and left kidney about * mm\n3. Otherwise, no significant abnormal findings'],
 ['SG',
  'Sonography of the liver, bile duct and pancreas was performed and showed: \nThe liver parenchyma shows homogeneous echogenicity without evidence of focal mass lesion.\nThe gall bladder shows small in size without evidence of stones.\nThe biliary tree is normal appearance without dilatation.\nThe pancreas shows normal appearance.\nThe spleen is normal appearance without enlargement.\nImpression:\n1.  Small size of gallbladder\n2.  Otherwise, no significant abnormal findings'],
 ['SCA',
  'Sonography of the abdomen was performed and showed: \nThe liver parenchyma shows homogeneous echogenicity without evidence of focal mass lesion but a calcification over * lobe.\nThe gall bl

In [34]:
df[df.content.map(lambda x: re.match(r'finding',x,flags=re.IGNORECASE)!= None)]

,code,content,date
1642,carhper,"Finding:\r\nSonography of the liver, bile duct...",2013-01-04
1697,bm20,Finding:\r\nBilateral mammogram: MLO and CC vi...,2014-02-07
1698,bm20o,Finding:\r\nBilateral mammogram: MLO and CC vi...,2014-02-07
1902,cilcspos,"Finding:\r\nSonography of the liver, bile duct...",2014-06-23
1929,dflgsspp,Finding:\r\nSonography of the abdomen was perf...,2014-01-24
2056,bmmwnlo,Finding:\r\nBilateral mammogram: MLO and CC vi...,2015-08-25
2057,bmmwno,Finding:\r\nBilateral mammogram: MLO and CC vi...,2015-08-25
2109,cietpo,"Finding:\r\nSonography of the liver, bile duct...",2013-09-25
2118,cihp,Finding:\r\nSonography of the abdomen was perf...,2013-10-15
2119,cihpc,Finding:\r\nSonography of the abdomen was perf...,2013-05-12


In [30]:
df.content

0                                                     Annn
1                                             Bil. apecies
2                                                     Bil.
3                                             Bil. apecies
4                                                both lung
5                                         Bil. lower lobes
6                                         BIL. upper lobes
7                                          central portion
8                                            inner portion
9                                            inner portion
10                                                    left
11                                           lower abdomen
12                                              left ankle
13                                      left lower abdomen
14                                         left lower lobe
15                                         left lower pole
16                                     left middle abdom